# Target

The main goal of this notebook is exercise two different file types (CSV and Parquet) and exercise the read, write and transform data process using Spark.

In [42]:
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [43]:
dir = "./data/ticket/"

os.listdir(dir)

['2022_Passagem.csv', 'ticket_parquet']

In [44]:
spark = SparkSession.builder.appName("file_types").getOrCreate()

In [45]:
file_name = "2022_Passagem.csv"

df = spark.read.csv(dir+file_name,header=True, sep=";", encoding="ISO-8859-1")

In [46]:
df.show(5)

+-----------------------------------+-------------------------+------------------+-----------------+---------------+-------------------+------------------+----------------+--------------------+-------------------+-----------------+---------------------+--------------------+------------------+----------------------+-----------------+---------------+----------------------+----------------------+
|Identificador do processo de viagem|Número da Proposta (PCDP)|Meio de transporte|País - Origem ida|UF - Origem ida|Cidade - Origem ida|País - Destino ida|UF - Destino ida|Cidade - Destino ida|País - Origem volta|UF - Origem volta|Cidade - Origem volta|Pais - Destino volta|UF - Destino volta|Cidade - Destino volta|Valor da passagem|Taxa de serviço|Data da emissão/compra|Hora da emissão/compra|
+-----------------------------------+-------------------------+------------------+-----------------+---------------+-------------------+------------------+----------------+--------------------+-------------

In [47]:
df.printSchema()

root
 |-- Identificador do processo de viagem: string (nullable = true)
 |-- Número da Proposta (PCDP): string (nullable = true)
 |-- Meio de transporte: string (nullable = true)
 |-- País - Origem ida: string (nullable = true)
 |-- UF - Origem ida: string (nullable = true)
 |-- Cidade - Origem ida: string (nullable = true)
 |-- País - Destino ida: string (nullable = true)
 |-- UF - Destino ida: string (nullable = true)
 |-- Cidade - Destino ida: string (nullable = true)
 |-- País - Origem volta: string (nullable = true)
 |-- UF - Origem volta: string (nullable = true)
 |-- Cidade - Origem volta: string (nullable = true)
 |-- Pais - Destino volta: string (nullable = true)
 |-- UF - Destino volta: string (nullable = true)
 |-- Cidade - Destino volta: string (nullable = true)
 |-- Valor da passagem: string (nullable = true)
 |-- Taxa de serviço: string (nullable = true)
 |-- Data da emissão/compra: string (nullable = true)
 |-- Hora da emissão/compra: string (nullable = true)



In [48]:
df_transformed = df.select([F.col(x).alias(x.replace("-","").replace(" ","_").replace("ç","c").lower()) for x in df.columns])

In [49]:
df_transformed.printSchema()

root
 |-- identificador_do_processo_de_viagem: string (nullable = true)
 |-- número_da_proposta_(pcdp): string (nullable = true)
 |-- meio_de_transporte: string (nullable = true)
 |-- país__origem_ida: string (nullable = true)
 |-- uf__origem_ida: string (nullable = true)
 |-- cidade__origem_ida: string (nullable = true)
 |-- país__destino_ida: string (nullable = true)
 |-- uf__destino_ida: string (nullable = true)
 |-- cidade__destino_ida: string (nullable = true)
 |-- país__origem_volta: string (nullable = true)
 |-- uf__origem_volta: string (nullable = true)
 |-- cidade__origem_volta: string (nullable = true)
 |-- pais__destino_volta: string (nullable = true)
 |-- uf__destino_volta: string (nullable = true)
 |-- cidade__destino_volta: string (nullable = true)
 |-- valor_da_passagem: string (nullable = true)
 |-- taxa_de_servico: string (nullable = true)
 |-- data_da_emissão/compra: string (nullable = true)
 |-- hora_da_emissão/compra: string (nullable = true)



In [50]:
df_transformed = df_transformed\
    .withColumn("valor_da_passagem",F.expr("cast(replace(valor_da_passagem,',','.') as float)"))\
    .withColumn("taxa_de_servico",F.expr("cast(replace(taxa_de_servico,',','.') as float)"))

In [51]:
df_transformed.printSchema()

root
 |-- identificador_do_processo_de_viagem: string (nullable = true)
 |-- número_da_proposta_(pcdp): string (nullable = true)
 |-- meio_de_transporte: string (nullable = true)
 |-- país__origem_ida: string (nullable = true)
 |-- uf__origem_ida: string (nullable = true)
 |-- cidade__origem_ida: string (nullable = true)
 |-- país__destino_ida: string (nullable = true)
 |-- uf__destino_ida: string (nullable = true)
 |-- cidade__destino_ida: string (nullable = true)
 |-- país__origem_volta: string (nullable = true)
 |-- uf__origem_volta: string (nullable = true)
 |-- cidade__origem_volta: string (nullable = true)
 |-- pais__destino_volta: string (nullable = true)
 |-- uf__destino_volta: string (nullable = true)
 |-- cidade__destino_volta: string (nullable = true)
 |-- valor_da_passagem: float (nullable = true)
 |-- taxa_de_servico: float (nullable = true)
 |-- data_da_emissão/compra: string (nullable = true)
 |-- hora_da_emissão/compra: string (nullable = true)



In [52]:
df_transformed.show(5)

+-----------------------------------+-------------------------+------------------+----------------+--------------+-------------------+-----------------+---------------+-------------------+------------------+----------------+--------------------+-------------------+-----------------+---------------------+-----------------+---------------+----------------------+----------------------+
|identificador_do_processo_de_viagem|número_da_proposta_(pcdp)|meio_de_transporte|país__origem_ida|uf__origem_ida| cidade__origem_ida|país__destino_ida|uf__destino_ida|cidade__destino_ida|país__origem_volta|uf__origem_volta|cidade__origem_volta|pais__destino_volta|uf__destino_volta|cidade__destino_volta|valor_da_passagem|taxa_de_servico|data_da_emissão/compra|hora_da_emissão/compra|
+-----------------------------------+-------------------------+------------------+----------------+--------------+-------------------+-----------------+---------------+-------------------+------------------+----------------+----

In [53]:
df_transformed.write.mode("overwrite").parquet("./data/ticket/ticket_parquet")

In [54]:
os.listdir(dir)

['2022_Passagem.csv', 'ticket_parquet']

In [62]:
print('Size of CSV file: ',os.path.getsize(dir+file_name),'\nSize of parquet file: ',os.path.getsize('./data/ticket/ticket_parquet'))

Size of CSV file:  68818335 
Size of parquet file:  8192


In [61]:
df_pqt = spark.read.parquet('./data/ticket/ticket_parquet')

df_pqt.printSchema()

root
 |-- identificador_do_processo_de_viagem: string (nullable = true)
 |-- número_da_proposta_(pcdp): string (nullable = true)
 |-- meio_de_transporte: string (nullable = true)
 |-- país__origem_ida: string (nullable = true)
 |-- uf__origem_ida: string (nullable = true)
 |-- cidade__origem_ida: string (nullable = true)
 |-- país__destino_ida: string (nullable = true)
 |-- uf__destino_ida: string (nullable = true)
 |-- cidade__destino_ida: string (nullable = true)
 |-- país__origem_volta: string (nullable = true)
 |-- uf__origem_volta: string (nullable = true)
 |-- cidade__origem_volta: string (nullable = true)
 |-- pais__destino_volta: string (nullable = true)
 |-- uf__destino_volta: string (nullable = true)
 |-- cidade__destino_volta: string (nullable = true)
 |-- valor_da_passagem: float (nullable = true)
 |-- taxa_de_servico: float (nullable = true)
 |-- data_da_emissão/compra: string (nullable = true)
 |-- hora_da_emissão/compra: string (nullable = true)

